In [4]:
import torch

# Custom loss function
**Input:** X and Y where each value is between -1 and 1. Both corresponding to the probability of home (+) vs away (-) where 0.5 = 50%, 1.0=100% home chance of winning, -0.7 = 70% visitor chance of winning etc.

Also the odds will be in the input as well

We will use Adam as the optimizer without passing in any. So by default smaller values are

1. Calculate the kelly criterion based on the predicted probability. When it suggests a negative number (no bet) set both X and y to 0
2. Calculate the pearson correlation coefficient, get its absolute value and multiply it by a hyperparameter constant. Combine this with the 



In [19]:
"""


x: 1d array of predictions between -1 and 1 where negative number means visitor predicted to win
y: ['H_Won', 'H_start_odds', 'V_start_odds']
pearson_multiplier: 
"""
def nfl_custom_criterion(x, y, pearson_multiplier=1.5):
    # ------------------------------------------------
    # Preliminary calculations
    # ------------------------------------------------
    y['decimal_odds'] = torch.where(x > 0, y['H_start_odds'], y['V_start_odds']) # Predicted vs actual odds (regardless of correct prediction)
    y_prob = 1 / y['decimal_odds']           # Probability (regardless of correct prediction)
    x_H_Won = torch.sigmoid(20 * x)          # Sigmoid so that it's differentiable. The 20 is arbitrarily large number
    y['correct_prediction'] = torch.sigmoid(x_H_Won * y['H_Won'])
    x = torch.abs(x)

    # ------------------------------------------------
    # 1. Calculate the kelly criterion
    # ------------------------------------------------
    kelly = x - ((1 - x) / y['decimal_odds'])

    # ------------------------------------------------
    # 2. Calculate the Pearson Correlation Coefficient
    # ------------------------------------------------
    n = x.size(0)
    sum_x = torch.sum(x)
    sum_y = torch.sum(y_prob)
    pow_x = torch.pow(x, 2)
    pow_y = torch.pow(y_prob, 2)
    x_mul_y = torch.mul(x, y_prob)
    sum_x_mul_y = torch.sum(x_mul_y, dim=0)

    # PCC Formula (eps to avoid NaN)
    eps = 1e-8
    pcc_numerator = n * sum_x_mul_y - sum_x * sum_y
    pcc_denominator_one = torch.sqrt(n * pow_x - sum_x**2 + eps)
    pcc_denominator_two = torch.sqrt(n * pow_y - sum_y**2 + eps)
    pcc = pcc_numerator / (pcc_denominator_one * pcc_denominator_two)
    pcc = pearson_multiplier * torch.abs(pcc)

    # ------------------------------------------------
    # 3. Penalize entirely wrong predictions
    # ------------------------------------------------
    wrong_penalty = -torch.log(x) * (1-x)
    wrong_penalty = torch.clamp(wrong_penalty, min=eps, max=10)
    return 1


def nfl_test(x, y, pearson_multiplier=1.5):
    # ------------------------------------------------
    # Preliminary calculations
    # ------------------------------------------------
    y['decimal_odds'] = torch.where(x > 0, y['H_start_odds'], y['V_start_odds']) # Predicted vs actual odds (regardless of correct prediction)
    y['probability'] = 1 / y['decimal_odds'] # Probability (regardless of correct prediction)
    x_H_Won = torch.sigmoid(20 * x)          # Sigmoid so that it's differentiable. The 20 is arbitrarily large number
    y['correct_prediction'] = torch.sigmoid(x_H_Won * y['H_Won'])
    print(y['correct_prediction'])

In [20]:
# Generate a tensor with random numbers between -1 and 1
amt_random_numbers = 10
x = torch.rand(amt_random_numbers) * 2 - 1
y = torch.rand(amt_random_numbers) * 2 - 1
# print(type(y))
# print(y.shape)
print(x)
print(y)

nfl_test(x,y)




tensor([-0.4204, -0.9837, -0.4733, -0.7989, -0.7495, -0.7199, -0.5978,  0.8713,
        -0.9945,  0.0550])
tensor([ 0.2483, -0.1146, -0.9051, -0.9363, -0.2433, -0.0987,  0.0338, -0.3846,
        -0.4936, -0.7511])


IndexError: too many indices for tensor of dimension 1